In [92]:
from nltk.grammar import DependencyGrammar
from nltk.parse.dependencygraph import DependencyGraph

In [93]:
import nltk
nltk.download('dependency_treebank')

[nltk_data] Downloading package dependency_treebank to
[nltk_data]     /Users/simone/nltk_data...
[nltk_data]   Package dependency_treebank is already up-to-date!


True

In [94]:
from nltk.corpus import dependency_treebank
t = dependency_treebank.parsed_sents()
print(t[0].tree())

(will
  (Vinken Pierre , (old (years 61)) ,)
  (join (board the) (as (director a nonexecutive)) (Nov. 29))
  .)


We can download the GloVe embeddings to vectorize our data.

**Uncomment to actually download!**

In [95]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

In [96]:
# We can now get a list of the input documents
import os
docs = os.listdir('data/dependency_treebank')
docs

['wsj_0095.dp',
 'wsj_0184.dp',
 'wsj_0177.dp',
 'wsj_0037.dp',
 'wsj_0126.dp',
 'wsj_0066.dp',
 'wsj_0052.dp',
 'wsj_0112.dp',
 'wsj_0003.dp',
 'wsj_0143.dp',
 'wsj_0153.dp',
 'wsj_0013.dp',
 'wsj_0102.dp',
 'wsj_0042.dp',
 'wsj_0076.dp',
 'wsj_0136.dp',
 'wsj_0027.dp',
 'wsj_0167.dp',
 'wsj_0194.dp',
 'wsj_0085.dp',
 'wsj_0007.dp',
 'wsj_0147.dp',
 'wsj_0056.dp',
 'wsj_0116.dp',
 'wsj_0180.dp',
 'wsj_0091.dp',
 'wsj_0122.dp',
 'wsj_0062.dp',
 'wsj_0173.dp',
 'wsj_0033.dp',
 'wsj_0023.dp',
 'wsj_0163.dp',
 'wsj_0072.dp',
 'wsj_0132.dp',
 'wsj_0081.dp',
 'wsj_0190.dp',
 'wsj_0106.dp',
 'wsj_0046.dp',
 'wsj_0157.dp',
 'wsj_0017.dp',
 'wsj_0006.dp',
 'wsj_0146.dp',
 'wsj_0057.dp',
 'wsj_0117.dp',
 'wsj_0181.dp',
 'wsj_0090.dp',
 'wsj_0123.dp',
 'wsj_0063.dp',
 'wsj_0172.dp',
 'wsj_0032.dp',
 'wsj_0022.dp',
 'wsj_0162.dp',
 'wsj_0073.dp',
 'wsj_0133.dp',
 'wsj_0080.dp',
 'wsj_0191.dp',
 'wsj_0107.dp',
 'wsj_0047.dp',
 'wsj_0156.dp',
 'wsj_0016.dp',
 'wsj_0094.dp',
 'wsj_0185.dp',
 'wsj_01

In [97]:
# And split train, dev, and test sets
import random
random.shuffle(docs)
train_docs = docs[:int(0.5*len(docs))]
dev_docs = docs[int(0.5*len(docs)):int(0.75*len(docs))]
test_docs = docs[int(0.75*len(docs)):]
len(train_docs), len(dev_docs), len(test_docs)

(159, 20, 20)

In [98]:
import numpy as np
def parse_dataset(docs, dir):
    """
    Parse the dependency treebank dataset.
    """
    X = []
    y = []
    for doc in docs:
        np_doc = np.loadtxt(dir+doc, str, delimiter='\t')
        X.append(" ".join(np_doc[:,0]))
        y.append(" ".join(np_doc[:,1]))
    return np.array(X),np.array(y)

In [99]:
X_train, y_train = parse_dataset(train_docs, 'data/dependency_treebank/')

In [100]:
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf

X_vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
X_train_ds = tf.data.Dataset.from_tensor_slices(X_train)
X_vectorizer.adapt(X_train_ds)
X_vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'of', 'to']

In [101]:
y_vectorizer = TextVectorization(max_tokens=100, output_sequence_length=200)
y_train_ds = tf.data.Dataset.from_tensor_slices(y_train)
y_vectorizer.adapt(y_train_ds)
y_vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'nn', 'in', 'nnp']

In [102]:
import numpy as np
embeddings_index = {}
embedding_dim = 100
with open("glove.6B."+str(embedding_dim) + "d.txt") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs
embeddings_index["the"]

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

In [103]:
vocabulary = X_vectorizer.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))
embedding_matrix = np.zeros((len(vocabulary)+2, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [104]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras import initializers

embedding_layer = Embedding(
    len(vocabulary)+2, # Number of tokens in the vocabulary
    embedding_dim, # Dimensions of the embedding
    embeddings_initializer=initializers.Constant(embedding_matrix),
    trainable=False,
)

In [110]:
from tensorflow.keras import layers, models, Input

model = models.Sequential()
model.add(embedding_layer)
model.add(layers.Bidirectional(layers.LSTM(32, return_sequences=True)))
model.add(layers.Dense(one_hot_depth, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 100)         919800    
                                                                 
 bidirectional_6 (Bidirectio  (None, None, 64)         34048     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, None, 38)          2470      
                                                                 
Total params: 956,318
Trainable params: 36,518
Non-trainable params: 919,800
_________________________________________________________________


In [112]:

model.fit(X_train, y_train, epochs=100, batch_size=128, val)

Epoch 1/100
2/2 [==============================] - 0s 114ms/step - loss: 0.4568 - accuracy: 0.0407
Epoch 2/100
2/2 [==============================] - 0s 92ms/step - loss: 0.4325 - accuracy: 0.0478
Epoch 3/100
2/2 [==============================] - 0s 94ms/step - loss: 0.4080 - accuracy: 0.0495
Epoch 4/100
2/2 [==============================] - 0s 88ms/step - loss: 0.3826 - accuracy: 0.0431
Epoch 5/100
2/2 [==============================] - 0s 148ms/step - loss: 0.3550 - accuracy: 0.0330
Epoch 6/100
2/2 [==============================] - 0s 166ms/step - loss: 0.3219 - accuracy: 0.0258
Epoch 7/100
2/2 [==============================] - 0s 90ms/step - loss: 0.2842 - accuracy: 0.0237
Epoch 8/100
2/2 [==============================] - 0s 84ms/step - loss: 0.2510 - accuracy: 0.0230
Epoch 9/100
2/2 [==============================] - 0s 121ms/step - loss: 0.2260 - accuracy: 0.0227
Epoch 10/100
2/2 [==============================] - 0s 97ms/step - loss: 0.2061 - accuracy: 0.0225
Epoch 11/100
2/

In [ ]:
y_train

array([array(['DT', 'NNP', 'NN', 'VBD', 'JJ', 'NNS', 'IN', 'DT', 'NN', 'IN',
              'JJR', 'IN', '$', 'CD', 'CD', 'IN', 'JJ', 'NN', 'IN', 'NNP', 'WDT',
              'VBZ', 'RB', 'IN', '$', 'CD', 'CD', 'IN', 'NN', 'CC', 'NN', 'NNS',
              'IN', 'JJ', 'CD', 'IN', 'NNS', 'IN', 'VBG', 'JJ', 'NN', 'CC', 'NN',
              '.', 'IN', 'DT', 'NNP', 'IN', 'NNP', 'NNP', ',', 'NNS', 'VBD', '$',
              'CD', 'CD', 'IN', 'JJ', 'NN', 'NNS', 'IN', 'DT', 'VBN', 'NN', 'NN',
              'NN', 'NN', ',', 'CC', 'JJ', 'NN', 'NNS', 'IN', 'DT', 'NNP', 'NNP',
              'NNP', 'NNP', 'VBP', 'VBN', 'IN', '$', 'CD', 'CD', 'IN', 'JJ',
              'CD', 'IN', 'NN', 'IN', 'DT', 'JJ', 'NNP', 'NN', '.', 'DT', 'NN',
              'VBD', 'IN', 'JJS', '$', 'CD', 'CD', 'IN', 'JJ', 'NN', 'CC', 'NN',
              'NN', 'IN', 'RB', ',', 'CC', 'IN', 'DT', 'NN', 'VBD', 'VBN', ',',
              'DT', 'NNS', 'VBP', 'VBN', 'TO', 'VBG', 'JJR', 'IN', '$', 'CD',
              'CD', 'IN', 'JJ', 'NN'

In [ ]:
vocab = X_vectorizer.get_vocabulary()
for word in X_train[0]:
    print(vocab[word], end=" ")

a housesenate conference approved major portions of a package for more than 500 million in economic aid for poland that relies heavily on 240 million in credit and loan guarantees in fiscal 1990 in hopes of stimulating future trade and investment for the agency for international development appropriators approved 200 million in secondary loan guarantees under an expanded trade credit insurance program and total loan guarantees for the overseas private investment corp are increased by 40 million over fiscal 1989 as part of the same poland package the conference approved at least 55 million in direct cash and development assistance as well and though no decision was made both sides are committed to adding more than 200 million in economic support funds and environmental initiatives sought by the bush administration the agreement on poland contrasts with the major differences remaining over the underlying foreign aid bill which has already provoked veto threats by the white house and is s